## Preprocessing

In [28]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [29]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(application_df[['EIN', 'NAME']], axis=1)

In [30]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []
app_type_vc = application_df['APPLICATION_TYPE'].value_counts()
df_val_counts = pd.DataFrame(app_type_vc)
df_value_counts_reset = df_val_counts.reset_index()
df_value_counts_reset.columns = ['unique_values', 'counts']

for x in range(len(df_value_counts_reset)):
  if (df_value_counts_reset['counts'].loc[x] < 400):
    application_types_to_replace.append(df_value_counts_reset['unique_values'].loc[x])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [31]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

classifications_vc = application_df['CLASSIFICATION'].value_counts()
df_val_counts = pd.DataFrame(classifications_vc)
df_value_counts_reset = df_val_counts.reset_index()
df_value_counts_reset.columns = ['unique_values', 'counts']

for x in range(len(df_value_counts_reset)):
  if (df_value_counts_reset['counts'].loc[x] < 1000):
    classifications_to_replace.append(df_value_counts_reset['unique_values'].loc[x])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [96]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_dummies = pd.get_dummies(application_df[['APPLICATION_TYPE','CLASSIFICATION','USE_CASE','SPECIAL_CONSIDERATIONS','AFFILIATION','ORGANIZATION']])
categorical_dummies.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,CLASSIFICATION_C1000,...,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1


In [97]:
 # Concatenate
application_dff = application_df[['STATUS','ASK_AMT','IS_SUCCESSFUL']]

preprocessed_application_df = pd.concat([application_dff, categorical_dummies], axis=1)
len(preprocessed_application_df.nunique())

35

In [98]:
# Dropping columns for Optimization
preprocessed_application_df = preprocessed_application_df.drop(preprocessed_application_df[['SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y','STATUS']], axis=1)
len(preprocessed_application_df.nunique())

32

In [99]:
# Split our preprocessed data into our features and target arrays
y = preprocessed_application_df['IS_SUCCESSFUL'].values
X = preprocessed_application_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [100]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [102]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=320, activation="relu", input_dim=31))
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            (None, 320)               10240     
                                                                 
 dense_57 (Dense)            (None, 80)                25680     
                                                                 
 dense_58 (Dense)            (None, 20)                1620      
                                                                 
 dense_59 (Dense)            (None, 1)                 21        
                                                                 
Total params: 37,561
Trainable params: 37,561
Non-trainable params: 0
_________________________________________________________________


In [103]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [104]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=40)

Epoch 1/40
804/804 [==============================] - 3s 2ms/step - loss: 0.5725 - accuracy: 0.7218
Epoch 2/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5642 - accuracy: 0.7249
Epoch 3/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5615 - accuracy: 0.7255
Epoch 4/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5609 - accuracy: 0.7276
Epoch 5/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5598 - accuracy: 0.7274
Epoch 6/40
804/804 [==============================] - 2s 3ms/step - loss: 0.5585 - accuracy: 0.7292
Epoch 7/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5586 - accuracy: 0.7289
Epoch 8/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5580 - accuracy: 0.7295
Epoch 9/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5577 - accuracy: 0.7291
Epoch 10/40
804/804 [==============================] - 2s 2ms/step - loss: 0.5572 - accuracy: 0.7296

In [105]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5553 - accuracy: 0.7261 - 496ms/epoch - 2ms/step
Loss: 0.5552871227264404, Accuracy: 0.726064145565033


In [106]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_trial_5.h5")